In [1]:

%matplotlib inline
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SimpleRNN

from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
import FinanceDataReader as fdr

Using TensorFlow backend.


In [23]:
pandf = fdr.DataReader("KS11","2009-01-01","2019-01-01")


In [24]:
#특성 추가 ------------------------------------------
    #이동평균선
def get_MA(df):
    MA_26=df["Close"].rolling(26).mean()
    MA_52=df["Close"].rolling(52).mean()
    df=df.assign(MA_26=MA_26,MA_52=MA_52).dropna()
    
    return df

    
    #스토캐스틱
def get_stochastic(df, n=15, m=5, t=3):
    # n일중 최고가
    ndays_high = df.High.rolling(window=n, min_periods=1).max()
    # n일중 최저가
    ndays_low = df.Low.rolling(window=n, min_periods=1).min()
 
    # Fast%K 계산
    kdj_k = ((df.Close - ndays_low) / (ndays_high - ndays_low))*100
    # Fast%D (=Slow%K) 계산
    kdj_d = kdj_k.ewm(span=m).mean()
    # Slow%D 계산
    kdj_j = kdj_d.ewm(span=t).mean()
 
    # dataframe에 컬럼 추가
    df = df.assign(kdj_k=kdj_k, kdj_d=kdj_d, kdj_j=kdj_j).dropna()
    
    return df
   
    #시간
def get_time(df):
    time=np.linspace(0,10,len(df),endpoint=False).reshape(-1,1)
    df=df.assign(time=time)
    
    return df

In [25]:
df=pandf
df=get_time(df)
df=get_stochastic(df)
df=get_MA(df)
# convert nparray

nparr = df.values
nparr.astype('float32')
print(nparr)

[[1169.95       1176.86       1176.86       ...   85.79334492
  1104.68038462 1134.385     ]
 [1161.81       1180.39       1186.91       ...   87.42346904
  1103.58923077 1134.46980769]
 [1170.94       1165.03       1180.38       ...   88.69249321
  1103.24692308 1134.41923077]
 ...
 [2028.01       2028.81       2037.83       ...   21.65334887
  2078.89038462 2083.74788462]
 [2028.44       2032.09       2035.57       ...   19.78398169
  2076.80807692 2081.50403846]
 [2041.04       2036.7        2046.97       ...   20.81083906
  2075.44230769 2079.5025    ]]


In [26]:
# normalization
scaler = MinMaxScaler(feature_range=(0, 1))
nptf = scaler.fit_transform(nparr)

In [27]:
# 값과 이평선 비교하기 ----------------

MA_26=df["Close"].rolling(26).mean()
y_before=MA_26.dropna()
df_before=df["Close"][25:]
y=np.where(df_before.shift(-1)>y_before.shift(-1),1,0)
X=nptf[25:]

In [28]:
# split train, test
train_size = int(len(nptf) * 0.7)
y_train2=y[:train_size]
y_test2=y[train_size:]
X_train2 = X[:train_size]
X_test2 = X[train_size:]



In [29]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train2, (X_train2.shape[0], 1, X_train2.shape[1]))
X_test = np.reshape(X_test2, (X_test2.shape[0], 1, X_test2.shape[1]))
y_train=y_train2
y_test=y_test2


In [30]:
# simple lstm network learning
model = Sequential()
model.add(LSTM(36, input_shape=(1, 12)))
for i in range(5):
    model.add(Dense(36,activation='sigmoid'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, epochs=10, batch_size=16,validation_split=0.1)
model.evaluate(X_test,y_test)

Train on 1526 samples, validate on 170 samples
Epoch 1/10
1526/1526 [==============================] - 1s 928us/step - loss: 0.6807 - acc: 0.5832 - val_loss: 0.7329 - val_acc: 0.3765
Epoch 2/10
1526/1526 [==============================] - 0s 321us/step - loss: 0.6775 - acc: 0.5885 - val_loss: 0.7611 - val_acc: 0.3765
Epoch 3/10
1526/1526 [==============================] - 1s 358us/step - loss: 0.5757 - acc: 0.6940 - val_loss: 0.4641 - val_acc: 0.8176
Epoch 4/10
1526/1526 [==============================] - 1s 344us/step - loss: 0.4000 - acc: 0.8453 - val_loss: 0.4255 - val_acc: 0.8412
Epoch 5/10
1526/1526 [==============================] - 0s 321us/step - loss: 0.3752 - acc: 0.8558 - val_loss: 0.4519 - val_acc: 0.8235
Epoch 6/10
1526/1526 [==============================] - 1s 334us/step - loss: 0.3616 - acc: 0.8571 - val_loss: 0.4104 - val_acc: 0.8294
Epoch 7/10
1526/1526 [==============================] - 1s 334us/step - loss: 0.3471 - acc: 0.8591 - val_loss: 0.3859 - val_acc: 0.8412
E

[0.3486134781796708, 0.8461538553237915]

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,np.where(model.predict(X_test)>0.5,1,0))

0.8461538461538461

In [32]:
model.predict(X_test)

array([[0.9278654 ],
       [0.93571407],
       [0.93581694],
       [0.9369302 ],
       [0.9421117 ],
       [0.944216  ],
       [0.9433894 ],
       [0.94905996],
       [0.95005584],
       [0.9501545 ],
       [0.94977236],
       [0.9435978 ],
       [0.9453496 ],
       [0.9471167 ],
       [0.9480032 ],
       [0.94318223],
       [0.9402962 ],
       [0.94296193],
       [0.94369423],
       [0.94495666],
       [0.94236004],
       [0.9453913 ],
       [0.9444331 ],
       [0.9343878 ],
       [0.9258256 ],
       [0.91832185],
       [0.93800473],
       [0.9396063 ],
       [0.9204211 ],
       [0.321481  ],
       [0.30028373],
       [0.10691547],
       [0.12130955],
       [0.12073094],
       [0.12699997],
       [0.11880219],
       [0.22313064],
       [0.89855254],
       [0.925594  ],
       [0.92791605],
       [0.9335811 ],
       [0.92166555],
       [0.94489753],
       [0.9410889 ],
       [0.9418351 ],
       [0.9459852 ],
       [0.9441158 ],
       [0.925

In [395]:
MA_26=df["Close"].rolling(26).mean()
MA_26=MA_26.dropna()
MA_26.shape


(713,)

In [396]:
pandf.shape

(789, 6)

In [397]:
MA_26.shape

(713,)

In [398]:
X_train2.shape

(516, 12)